In [ ]:
# This cell reads in the downloaded stuff and creates a single uncompressed zarr containing all of the given data so that we can easily use it later.
#TODO: xarray is bottleneck during training since its too slow (single-threaded loading) so all the shit deadlocks, we should make that better by decompressing to numpy arrays.
import xarray as xr
from ocf_blosc2 import Blosc2
# sat-nonhrv
sat = xr.open_dataset(
    "/data/satellite-nonhrv/2021/1.zarr.zip", engine="zarr", chunks="auto"
)
for month in range(2, 13):
    sat2 = xr.open_dataset(
        f"/data/satellite-nonhrv/2021/{month}.zarr.zip", engine="zarr", chunks="auto"
    )
    sat = xr.concat((sat, sat2), dim="time")
sat = sat.chunk(chunks={"time": "auto"})
sat.to_zarr("/data/satellite-nonhrv_concat/2021.zarr")
# weather
NWP_FEATURES = [
    "t_500", "clcl", "alb_rad", "tot_prec", "ww",
    "relhum_2m", "h_snow", "aswdir_s", "td_2m", "omega_1000"
]

datasets = []
for month in range(1, 13):
    dataset = xr.open_dataset(
        f"/data/weather/2021/{month}.zarr.zip", engine="zarr", chunks="auto"
    )
    dataset = dataset[NWP_FEATURES]
    datasets.append(dataset)
data = xr.concat(datasets, dim="time")
data = data.chunk(chunks={"time": "auto"})
data.to_zarr("/data/weather_concat/2021.zarr")
